> # 0. 포트폴리오 구성
1. 인텔을 이기다
    - Intel AI Developer Program 실습 도중 보다 높은 예측정확도를 산출할 수 있는 지표를 발견하기까지의 과정
2. 배민라이더스 앱리뉴얼 효과 분석 제안서
    - 배민라이더스 라이더용 앱의 리뉴얼을 앞두고 있다는 가정 하에 그 효과에 대한 사전 검증을 위한 분석 계획 수립

> # 1. 인텔을 이기다

## [인텔 AI 개발자 프로그램] 머신러닝 2주차 - 지도학습과 KNN

이번 주 실습에서는 통신사의 고객이탈율 데이터를 활용할 거예요. 이름은 `Orange_Telecom_Churn_Data.csv`입니다. 함께 데이터를 불러오고, 필요한 정제를 하고, 최종적으로 KNN 모델을 활용하여 각 고객별 특성에 따른 이탈 여부를 예측하고자 합니다.

### 문제 재정의 (구체화)
    1. 목표: 오렌지텔레콤을 위한 고객이탈 여부 예측
    2. 데이터: 고객 기본정보 및 과거 이탈 여부
    3. 제한: KNN을 활용하여 예측할 것

In [ ]:
# 필요환경 구성

from __future__ import print_function
import os

data_path = ['../input/intelml101class1']

## 문제 1

* 먼저 데이터를 불러오세요. 이후 지표(column)들과 데이터를 살펴보세요.
* 지표 중 주 이름(state), 지역번호(area code), 전화번호(phone number)가 포함되어 있네요.
머신러닝 모델을 설계하는 데 이것들이 좋은 지표가 될 수 있을까요? 여러분의 의견과 이유를 알려주세요.
* 우리는 이걸 사용 안 할 거예요. 그러니 지표 목록에서 삭제해도 되겠죠.

### 우리 방법

1. 데이터 불러오기

In [ ]:
import numpy as np
import pandas as pd

filepath = os.sep.join(data_path + ['Orange_Telecom_Churn_Data.csv'])
print(filepath)
data = pd.read_csv(filepath)

# 마지막의 예측목표열(churned: 고객이탈)을 포함해 총 21개 지표가 있음 / 사례 고객: 5,000명
data.tail(n=1)

2. 지표 살펴보기

In [ ]:
# 기본적 통계수치 확인
data.describe()

In [ ]:
# 지표 이름으로 내용 추측
data.columns

In [ ]:
# 실제 내용 확인
data.tail(n=1)

> 지표 및 데이터 기초 분석
1. 타겟: 이탈(True/False), 마지막 열에 있음
2. 열 구성: 총 21 / 타겟열 제외 20
    - 수치: 14 / 분류: 1 (state) / True or False: 3 (intl_plan, voice_mail_plan / 타겟 제외: 2)
    - 숫자지만 수치는 아닌 열: 3 (account_length, area_code, phone_number)
        - 인덱스를 활용하면 되므로 account_length, phone_number는 삭제 > 삭제 전 인덱스로 계정 접근 가능한지 확인
        - state에 위치 정보가 포함되어 있으므로 area_code도 삭제

In [ ]:
# 인덱스로 계정 접근 가능 > account_length 삭제
data.iloc[-1,:]

In [ ]:
## 필요없는 열 삭제
col_unused = ['account_length', 'area_code', 'phone_number']
data_slim = data.drop(columns=col_unused)
print(len(data_slim.columns)) # 18 = 21 - 3
data_slim.tail(n=1)

### Intel 방법

In [ ]:
import pandas as pd

# 데이터 불러오기
filepath = os.sep.join(data_path + ['Orange_Telecom_Churn_Data.csv'])
data = pd.read_csv(filepath)

In [ ]:
data.head(1).T

In [ ]:
# 필요없는 열 삭제
data.drop(['state', 'area_code', 'phone_number'], axis=1, inplace=True)

In [ ]:
data.columns

### 차이와 배움
1. 인텔: data.head(1).T - 트랜스포즈를 사용하여 전체 컬럼을 보기 쉽게 만듦
2. drop(inplace=True) 무슨 기능?
    - True일 때, 드롭된 것을 뺀 새 데이터가 기존 데이터를 대체함
    - False일 때, 복제된 새 데이터를 얻을 수 있음 (기존 것 유지)
    - False가 기본 값

## 문제 2

* 열 중 일부는 분류형 값이고 일부는 소숫점 숫자입니다. 수업 중 다룬 방식 중 하나를 적용하여 이 지표들을 수치화하세요.
* KNN 모델은 단일화된 수치적 데이터를 요구한다는 거 기억하시죠? 마찬가지로 수업 내 방식을 적용해서 데이터를 변환하세요.

### 우리 방법

1. 분류를 숫자로 바꾸기

In [ ]:
# 현위치 파악
data_slim.tail(n=1).T

In [ ]:
# 전체 열 데이터 종류 확인
pd.DataFrame(data_slim).dtypes

수치형 제외 데이터 종류 (위에서 1차 파악한 내용을 갱신)
1. 분류형: 3 (state, intl_plan, voice_mail_plan)
    - intl_plan, voice_mail_plan은 bool형이 아니라 분류형으로 확인
2. bool형: 1 (churned)
* 숫자지만 범주에 가까운 열: number_customer_service_calls (서비스콜 수 범주화 가능) > 다른 열 변환 후 변환 필요 확인

In [ ]:
# 실험적으로 State부터 변환해보기 / factorize 활용

# 변환 전
print(data_slim.state.tail(n=3))
print()

# 변환 후
data_slim.state = pd.factorize(data_slim.state)[0]
print(data_slim.state.tail(n=3))
data_slim.state.unique()

In [ ]:
# 나머지 비숫자 열 변환
for i in range(len(data_slim.columns)):
    if data_slim.dtypes[i] != int and data_slim.dtypes[i] != float: # 숫자 여부 확인
        col = data_slim.columns[i]
        data_slim[col]= pd.factorize(data_slim[col])[0]
data_slim.tail(n=3)

In [ ]:
# 변환 결과
data_slim.dtypes

> #### 참고 학습: 객체(object)에 대하여
1. intl_plan / voice_mail_plan: bool이 아닌 객체(우리의 경우 분류형)
2. 객체 데이터 유형이란?
    - 객체지향언어에서 비객체형 값을 감싸서 동적 객체처럼 보이게 만드는 데이터유형
3. 동적 객체란?
    - (사전에 정의되어 프로그램 실행 내내 존재하는) 정적 객체의 반대
    - 예를 들어 동적 객체를 활용하여 일단 "물의 양"이라는 클래스를 정의내리면, 이후에 사용자가 새 박스에 새 물을 채울 때엔 새로운 예시를 동적으로 만들어낼 수 있음 
    - 정적 객체는 미리 이런 상황을 예측하여 사전에 그 새로운 물의 양까지 정의내려야 함
4. 하지만 intl_plan / voice_mail_plan은 오직 Y/N 두 값만 갖고 있으므로 위 필요에 맞지 않는 것으로 보임
5. state 역시 50개라는 제한된 입력값을 갖고 있음
6. 아무래도 아직 '객체'에 대해 분명한 그림이 그려지지 않은 듯함 (객체지향언어에 대해서도)
7. 그때그때 나올 때마다 좀 더 찾아보며 배워야함
8. 이제 위의 두 열이 bool이 아닌 객체인 걸 알았으니 변환 기능을 새로 써봅시다 (최초 버전: bool 여부 확인 후 변환)
    - bool 혹은 객체(분류형) 여부에 상관없이 변환하도록 코드 변경 완료 (숫자 여부 확인으로 일반화)

In [ ]:
# 추가적으로, 숫자지만 범주에 가까운 열 (number_customer_service_calls) 확인

# 변환 전
print(data_slim.number_customer_service_calls.unique())

# 변환 가능해보임 > 해보자 > 실은 변환할 필요없었음 > 이미 숫자임!
data_slim['number_customer_service_calls'] = pd.factorize(data_slim.number_customer_service_calls)[0]

# 변환 후
print(data_slim.number_customer_service_calls.unique())

수치 단일화(scaling) 전 고려 사항
    - 수업 중 배운 세 가지 스케일링 중에서 이 문제에는 뭐가 최적일까? (각 값의 실제 계산과정 확인)
        - standard
            x_scaled = (x-mean) / standard deviation
        - min-max
            x_scaled = (x-min(x)) / (max(x)–min(x)) *기본 설정 범주 = [0,1]
        - max-abs
            x_scaled = x / max(abs(x))
        
        
- 아직 이 문제에 뭐가 최적인지 모르겠음 > 일단 "표준"인 standard로 실험

2. 수치 단일화 (scaling)

In [ ]:
# StandardScaler
from sklearn.preprocessing import StandardScaler

"""
잠깐 메모: 파이썬식 변수 이름 짓기 (검색 후 적용)
> 계속 아래_막대_방법을 적용하자,
  작업하고 있는 환경이 엄격하게 camelCase를 따르는 게 아니면
"""
StdSc = StandardScaler()
data_slim_std_scaled = pd.DataFrame(StdSc.fit_transform(data_slim), columns = data.columns)

data_slim_std_scaled.describe().T.tail(n=3)

In [ ]:
# 타겟 확인 (스케일링 전) > 0/1 맞음
data_slim.churned.tail(n=5)

In [ ]:
# 타겟 확인 (스케일링 후) > 큰일 남!
data_slim_std_scaled.churned.tail(n=5)

#### 중간분석: 0과 1이 사라진 이유

1. 원인은 StandardScaler 계산식에 있음
    standardized x = (x-mean) / standard deviation
    - 우리 데이터 타겟값의 mean은 0과 1 사이 소숫점 숫자 중 하나임 (전부 0이거나 전부 1이 아닌 한)
    - 그러니 그 숫자를 각 x에서 빼주고 standard deviation으로 나누면 0과 1 이외의 값이 나올 수 있음 (음수 역시 가능)
2. 즉, 우리 문제에 최적의 스케일러가 뭔지는 몰라도 적어도 StandardScaler는 아님
3. 수치화 이후 타겟값이 분류를 위해 사용할 수 없는 형태가 되기 때문
4. 물론 churned열을 빼고 스케일링을 적용하면 문제 없음
5. 나머지 스케일러 적용 후엔 어떤지 확인

In [ ]:
# MinMaxScaler (기본 범주인 (0,1) 적용)
from sklearn.preprocessing import MinMaxScaler

#MMSc = MinMaxScaler(feature_range=(0,3)) # 스케일링 범주에 따른 정확도 차이 확인용
MMSc = MinMaxScaler()
data_slim_MMSc_scaled = pd.DataFrame(MMSc.fit_transform(data_slim), columns = data.columns)

data_slim_MMSc_scaled.describe().T.tail(n=3)

In [ ]:
# MaxAbsScaler
from sklearn.preprocessing import MaxAbsScaler

MASc = MaxAbsScaler()
data_slim_MASc_scaled = pd.DataFrame(MASc.fit_transform(data_slim), columns = data.columns)

data_slim_MASc_scaled.describe().T.tail(n=3)

중간분석: MinMax와 MaxAbs가 같은 결과를 내놓는 이유
1. 역시 원인은 계산식에 있음
    - min-max: x_scaled = (x-min(x)) / (max(x)–min(x))
    - max-abs: x_scaled = x / max(abs(x))
2. 두 계산식이 일치하는 경우는 min(x)==0 and max(x)==max(abs(x))
    - 즉 각 열의 값이 0 또는 양수이고 그 최소값이 0일 때
3. 오렌지텔레콤 데이터의 경우 이 사례에 맞음 (아래 참고)
    - 그러므로 MinMax와 MaxAbs 스케일링 후 얻는 결과값이 같음
    - 단, MinMaxScaler의 기본 범주인 [0,1]을 변경하지 않았을 때만 그러함

In [ ]:
# 수치화 후 스케일링 전 데이터 min/max 확인 > 모든 열 >= 0, 모든 min = 0
data_slim.describe().T

### Intel 방법

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

for col in ['intl_plan', 'voice_mail_plan', 'churned']:
    data[col] = lb.fit_transform(data[col])

In [ ]:
# sklearn 경고 끄기
import warnings
warnings.filterwarnings('ignore', module='sklearn')

from sklearn.preprocessing import MinMaxScaler

msc = MinMaxScaler()

data = pd.DataFrame(msc.fit_transform(data),
                    columns=data.columns)
data.describe().T.tail(n=3)

### 차이와 배움

1. sklearn LabelBinarizer(인텔) VS. pandas factorize(우리)
    - LabelBinarizer는 정수를 / factorize는 소숫자리 값을 줌
    - 스케일링을 뒤이어 하면 둘 중 뭘 써도 좋음, 스케일링 후엔 어차피 소숫자리 값이 됨
    - 스케일링을 하지 않을 거라면 LB가 더 나은 선택, 분류를 표현하기에 더 좋기 때문
    > 타겟열(chunred)를 좀 더 살펴본 뒤
        - 이 경우엔 LB만 쓸 수 있음
        - 안 그러면 0/1이 소숫자리로 바뀌어버림 (위에서 본 것처럼!)
        - 물론 이건 분류에 쓸 수 없음
        - LB 활용해서도 해보기 (완료)
2. 인텔: 세 가지 열을 변환하기 위해 for loop 하나를 만들기
    - 가능한 모든 걸 자동화 해보자는 태도가 필요
    - 머리를 바쁘게, 손은 한가롭게
    - 우리도 loop 활용해서 변환 (완료)
3. 적은 게 많은 것
    - dataframe화 하는 단계랑 fit_transform 단계를 한 줄로 합쳐보자 (완료)

* 잠깐 실험실

In [ ]:
# LabelBinarizer로 변환해보기

# 실험용 데이터 복사
data_lb = data_slim.copy()

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()

for column in ['intl_plan', 'voice_mail_plan', 'churned']:
    data_lb[column] = lb.fit_transform(data[column])

data_lb.tail(n=1).T

In [ ]:
# 타겟 변환 여부 상세 확인
data_lb.churned

* 실험 종료

## 문제 3

* 전체 열을 타겟과 트레이닝 데이터로 나누세요. 두 개의 테이블이 나오도록요.
* 위 데이터와 k=3 값을 적용하여 KNN 모델을 학습시키세요. 같은 트레이닝 데이터를 활용해 타겟값을 예측하세요.

### 우리 방법

1. 데이터를 X, y로 나누기

In [ ]:
# 현위치 파악
print(data_slim_MMSc_scaled.columns) # 그나저나 이 변수 이름이 너무 길지 않음?;
data_slim_MMSc_scaled['churned'].tail(n=3)

In [ ]:
# 위에서 배운 것(drop)부터 적용해보기
y_MMSc = data_slim_MMSc_scaled['churned'] # 이름 짧게
X_MMSc = data_slim_MMSc_scaled.drop(columns='churned')

print(y_MMSc.tail(n=3).T)
X_MMSc.tail(n=3).T.tail(n=3)

2. KNN 학습시키기 / 이탈 여부 예측하기

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier(n_neighbors=3)
KNN = KNN.fit(X_MMSc, y_MMSc)

y_predict = KNN.predict(X_MMSc)
print(y_predict)

# 한 뼘 더: KNN에 탑재된 score 기능을 활용해서 정확도 측정
accuracy = KNN.score(X_MMSc, y_MMSc)
print(accuracy)

#### 결과값 분석

Q. 94.44% - 오버핏이 아니라는 증거?
- 학습한 데이터와 예측에 쓰인 데이터가 같지만, 실제값과 다른 예측이 나오는 이유
    1. 2주차 학습자료 중 k=1 VS. k=all 비교 그래프를 다시 볼 것 (완료)
    2. k=3는 가장 가까운 3개의 다른 사례를 뺀 나머지는 분류에 활용하지 않음을 의미
    3. 거꾸로 얘기하면 본인을 제외한 3개의 다른 사례를 본인의 이탈 여부를 예측하는 데 참고하겠다는 뜻
    4. 이렇게 볼 때 94.44% 정확성(실제값과 5.56% 차이)은 낮지 않음
    5. 실제로 KNN에서는 k의 크기가 오버핏/언더핏과 관련이 있음 (3주차에서 집중적으로 살펴볼 내용)



### Intel 방법

In [ ]:
# 타겟을 제외한 열 목록
x_cols = [x for x in data.columns if x != 'churned']

# 데이터 두 개로 나누기
X_data = data[x_cols]
y_data = data['churned']

# # 다른 방법:
# X_data = data.copy()
# y_data = X_data.pop('churned')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn = knn.fit(X_data, y_data)

y_pred = knn.predict(X_data)
y_pred

### 차이와 배움
1. 인텔: 타겟을 뺀 나머지 열을 찾기 위해 for loop 활용
    - 이번에는 우리 방식(drop)이 더 나은 듯함
    - 하지만 인텔의 2번째 방법(pop)이 더 짧고 좋아보임 > 해보자! (완료)
2. 정확도 측정은 다음 문제에서 할 거였는데 미리 해버림. 그럼 아래에서는 직접 기능을 짜보기로 함

- 잠깐 실험실

In [ ]:
# pop으로 데이터 쪼개기
popped_X = data.copy() # 원래 데이터는 살려두기
popped_y = popped_X.pop('churned')

print(popped_y.tail(n=3))

popped_X.tail(n=3).T.tail(n=3)

- 실험 종료

## 문제 4

아직 오류 측정 방법은 안 다뤘죠? 그래도 정확도는 이해하기 쉽습니다. 단순히 전체 중 맞춘 예측의 비율을 계산하는 거니까요.

* 실제 타겟값과 예측값을 활용해 정확도를 측정하는 기능을 만들어보세요.
* 이 기능과 현재 데이터로 직접 만든 KNN 모델의 정확도를 계산하세요.

### 우리 방법

1. 정확도 측정 기능 만들기

In [ ]:
# 정확도 측정기
def get_accuracy(prediction, target):
    return sum(prediction == target) / float(len(prediction))

get_accuracy(y_predict, y_MMSc)

### Intel 방법

In [ ]:
# 맞는 예측 비율 계산기

def accuracy(real, predict):
    return sum(y_data == y_pred) / float(real.shape[0])

In [ ]:
print(accuracy(y_data, y_pred))

### 차이와 배움
1. 아 참, 측정 기능을 만들라고 했지 (처음엔 1회성 for loop으로 결과만 얻음)
    - 당장 고치자 (완료)
    - 항상 생각하자 '범용성' '확장성' '재활용성'
2. 94.22% (인텔) VS. 94.44% (우리)
    - 인텔과 우리 모두 같은 스케일링(minmax)과 데이터를 사용
    - 그런데 정확도는 우리 모델이 0.22%p 높음
3. 좀 더 들어가보자

- 잠깐(으로 계획했으나 하루를 넘기고만) 실험실

In [ ]:
# 1. 아마 정확도 측정기에 차이가 있지 않을까
# 두 데이터(인텔, 우리)를 한 측정기(우리)에 적용해보자
print(get_accuracy(y_data, y_pred), get_accuracy(y_MMSc, y_predict))

In [ ]:
# 측정기 차이 없음

In [ ]:
# 2. 두 예측값 자체가 얼마나 다른가
get_accuracy(y_pred, y_predict)

In [ ]:
# 어느 정도 다르긴 하지만, 이 한 수치만으론 원인 파악이 힘듦 (인텔/우리 각각의 정확도와도 다름)

In [ ]:
# 3. 고객이탈(1)로 예측된 값을 모아보자 (총 5000개 사례 중 얼마나 되는지)
print(sum(y_pred), sum(y_predict)) # 예측값
print(sum(y_data), sum(y_MMSc)) # 실제값

In [ ]:
# 이탈 고객 수 총합만으론 개별 고객 예측값과 실제값의 일치 여부를 알 수 없음

In [ ]:
# 4. 각 예측값이 각 실제값과 얼마나 같은가
print(sum(y_pred==y_data), sum(y_predict==y_MMSc))

In [ ]:
# 위의 수치로 볼 때 인텔보다 우리 모델이 11명 더 많은 고객에 대해 정확한 예측을 내놓는 것을 확인
# 하지만 여전히 이 차이의 '원인'을 모르겠음

.
.
.
.
.
.
.
.
.
불면과 자기환멸의 밤이 지나고
.
.
.
.
.
.
.
.
.

찾았다! (다음날 낮에 전체 과정을 처음부터 다시 살펴보던 중 원인을 발견)

In [ ]:
# 우리는 트레이닝 데이터에서 'account_length'를 삭제
# 이 열이 가질 수 있는 값의 종류는 218 가지 / 각 값이 갖는 평균 고객의 수는 약 23명
print(len(data.account_length.unique()), len(data)/len(data.account_length.unique()))

In [ ]:
# 반면 인텔은 'state'를 필요없다고 보고 삭제
# 이 열이 가질 수 있는 값의 종류는 51 가지 / 각 값이 갖는 평균 고객의 수는 약 98명
print(len(data_slim.state.unique()), len(data_slim)/len(data_slim.state.unique()))

# 최종분석: 인텔 모델과 우리 모델의 정확도 차이와 그 원인

## Q1. 예측에 필요한 지표를 선택하는 기준
1. 우리는 state보다 account_length가 더 필요치 않은 정보라고 판단하고 삭제
2. 그 결과 0.22% 높은 정확도를 얻음 (같은 k=3 값, min-max scaler를 적용한 KNN모델을 사용)
    - 실제로 min-max scaler의 범위가 같지는 않았음 (이에 대한 실험은 아래 참고)
3. 이는 고객 이탈을 예측하는 데 state가 account_length에 비해 0.22% 도움되는 정보를 지니고 있다는 의미
4. 위에서 확인했듯 state의 한 값이 품을 수 있는 고객의 수는 account_length에 비해 4배 이상 (98:23)
    - 이탈 고객(T/F 두 가지)을 분류하는 범주로서 state가 account_length에 비해 더 포괄적이고 적절하다고 볼 수 있음
5. 실제로 같은 지역 내 고객들은 생활수준, 브랜드선호도, 심지어 정치성향에 이르기까지 공통점을 보이곤 함
    - 오랜지텔레콤의 통신망을 활용하는 고객들 역시 예외는 아닐 것 (통신사를 옮기는 성향도 지역에 반영 가능)
6. 반면 계정의 길이는 이런 성향을 반영하는 지표로 기능한다고 기대하기 힘듦 (각 고객이 개별적으로 설정한 값이고, 그 값 자체도 아닌 길이이므로)
    - 고객이 만든 통신사 계정의 길이와 이탈율 사이에 상관관계나 심지어 인과관계를 밝힌다면 이는 통신업계(및 통계학계)에 일대 혁신일 것
    - 우리 사례의 경우, 계정 길이가 주 이름이라는 위치 정보에 비해 예측에 기여하는 정도가 떨어진다는 것이 위 수치로 증명됨
    - 그럼에도 계정 길이 지표 자체에 대한 관심과 투자 대비 성과가 기대된다면 추가 실험 및 분석 가능

결론: 예측 지표로서 state와 account_length 중 하나를 활용해야 한다면 선택은 state가 맞음 (위치 정보를 포함하고 있으므로)

## Q2. min-max 스케일러의 범위가 정확도에 끼치는 영향
1. 최초 우리는 [0,3]으로 범위를 설정 > 예측정확도 94.44% 도출
2. Intel 은 기본값인 [0,1]을 사용 > 예측정확도 94.22% 도출
3. 모든 변인을 동일하게 두고 우리 모델의 스케일링 범주만 [0,1]로 바꾼 뒤 정확도 재측정 (현재 모델에 적용됨)
     - 여전히 94.44%의 정확도를 보임 / 인텔 모델 역시 94.22%로 그대로
     
결론: min-max 스케일러의 범위는 정확도에 영향을 주지 않음

- 실험 종료

In [ ]:
# 잠시 쉬는 줄 (이건 읽지 마시고 눈을 쉬게 해주세요!)

>  # 2. 배민라이더스 앱리뉴얼 효과 분석 제안서

## [상황]

- 현재 우아한청년들은 라이더들이 주문을 받고 배달을 하는 데 활용하는 '배민라이더스' 앱의 리뉴얼을 계획 중입니다.
- 목표는 배달서비스 품질 개선이며 이를 위한 여러분의 제안을 기다립니다.

## [용어 정의 및 구체화]

1. 라이더
	- 배민라이더스에 소속되어 배달 임무를 수행하는 자 
	- 그 중에서도 배달 임무에 배민라이더스 앱을 적극 확용하는 자
	- 적극: 모든 배달 주문을 배민라이더스 앱을 통해 받고 처리하는 자
	- 비적극: 비공식인 경로를 통한 주문을 받아 수익을 창출하는 자
	- 비공식: 배민라이더스 앱 이외의 모든 경로 (다른 앱, 문자, 전화 등)
2. 배민라이더스 앱
	- 우아한형제들 개발팀을 통해 개발/배포된 배민라이더스 라이더용 배달 지원 어플리케이션
	- 비공식 앱: 앱스토어, 플레이스토어 등이 아닌 비공식 경로를 통해 개발/배포된 유사 앱
	- 비공식 경로: 개발자 개인 배포, 애플 탈옥 마켓을 통한 배포 등
3. 리뉴얼
	- 현재 버전에 속하지 않는 기능/외형을 포함하는 어플리케이션 갱신 활동
	- 현재 버전에 속하지 않는 기능/외형이라도 과거 기 배포된 버전과 전체적으로 동일한 조합이라면 리뉴얼에 속하지 않음
	- 전체가 아닌 부분은 과거의 기능/외형을 참고 및 활용할 수 있음
4. 배달서비스 품질
	- 정량적: 
            - 주문 수
                - 라이더 별 주문 수, 배민라이더스 전체 주문 수
                - 우아한청년들 서비스(B마트 포함) 전체 주문 수, 우아한형제들 서비스(배달의민족 포함) 전체 주문 수
            - 위험 요소
                - 배달 소요 시간, 배달 중 사고 발생 비율 
		    - 만족도 수치
                - 앱스토어/플레이스토어 고객 리뷰(5점 만점), 고객 리뷰 수, 이 중 긍정/부정 리뷰의 비율
		    *위 모든 지표의 측정 및 평가는 30일을 기준으로 함(변동 가능)
	- 정성적: 
            배민라이더스 고객 만족도 설문조사, 미스터리오더(미스터리쇼퍼의 배달 버전)를 통한 항목별 평가 취합
5. 개선
	- 주문 수 및 긍정 리뷰 수: 정량적 지표별 기존 수치 대비 10% 이상의 증가
	- 배달 시간 및 사고 비율: 정량적 지표별 기존 수치 대비 10% 이상의 감소
	- 10% 미만의 변화: 계절성, 이벤트성, 일반적인 증감 주기 등에 따른 변화로 간주
	- 정성적 지표: 정량적 지표 개선에 대한 참고자료로 활용
	- 예외: 전체 정성적 의견의 50% 이상이 부정적일 경우 정량적 개선이 있더라도 서비스 품질 전체는 개선되지 않았다고 판단

## [목표 재정의]

- 우아한청년들은 배달서비스 품질 평가 항목 중 최소 1개 이상의 개선을 위해
- 배민라이더스 공식 앱의 기능적/외형적 변화를 계획하고 있습니다. 
- 앱의 사용자는 배민라이더스 소속 공식 라이더들이며
- 배달 주문을 받고 배달 경로를 확인하는 등 전체 배달 프로세스에 활용합니다.

## [문제 파악]

1. 리뉴얼이 필요한가?
2. 필요하다면, 그 목적은 없는 기능의 추가인가? 있는 기능의 강화인가?
3. 필요하지 않다면, 리뉴얼을 하지 않고 배달서비스 품질 개선을 이룰 수 있는 방법은 무엇인가?

## [문제 구체화]

1. 리뉴얼의 필요성
	- 현재 앱은 얼마나 만족스러운가?
	- 과거 버전의 앱과 비교하면 얼마나 더/덜 만족스러운가?
	- 경쟁 배달업체의 앱과 비교하면 얼마나 더/덜 만족스러운가?
	- 일반 지도 앱과 비교하면 얼마나 더/덜 만족스러운가?
	- 라이더들이 평소 사용하는 다른 앱과 비교하면 얼마나 더/덜 만족스러운가?
2. 리뉴얼의 목적
	- 필요하지만 없는 기능은 무엇인가?
	- 있지만 쓰이지 않는 기능은 무엇인가?
	- 쓰이지만 제대로 작동하지 않는 기능은 무엇인가?
	- 앱 다운을 일으키는 기능은 무엇인가?
	- 외형적인 부분에서 우아한청년들, 배달의민족 전체 브랜드와 얼마나 조화로운가?
3. 리뉴얼 없이 개선
	- 앱의 홍보가 덜 된 것은 아닌가?
	- 라이더가 앱을 활용하는 데 따른 보상 혹은 의무가 없거나 부족하진 않은가?
	- 앱을 통해 얻은 데이터를 앱 자체가 아닌 다른 서비스에 활용할 수는 없는가?
	- 다른 서비스를 통해 얻은 데이터를 배민라이더스 앱에 활용할 수는 없는가?
	- 위의 방법이 있고 그로 인한 서비스 개선이 기대된다면, 그 적용을 위해 리뉴얼이 필요한가? 데이터 연결을 통해 구현할 수는 없는가?

## [필요 자원/자료 정의]

1. 리뉴얼의 필요성
	- 현재/과거 앱 만족도 조사
        - 앱스토어/플레이스토어 리뷰 선호도 분석
        - 라이더 설문조사
	- 경쟁 배달업체 앱 만족도 조사
        - 앱스토어/플레이스토어 리뷰 선호도 분석
        - 경쟁업체 라이더용 앱 활용 후 결과 정리 (등록없이 앱만 사용 가능한지 파악)
        - 등록이 필요하다면 배민라이더 중 해당 앱 활용 가능한 라이더 파악
        - 없다면 외부 라이더 중 해당업체에 소속되지 않았으나 앱 활용 가능한 라이더 탐색 후 설문 의뢰
	- 일반 지도 앱 만족도 조사
        - 앱스토어/플레이스토어 리뷰
        - 배민라이더스 소속 라이더 중 일부(30명 내외)를 선별하여 배달 시 일반 지도 활용 후 결과 정리
	- 일반 앱 만족도 조사
        - 앱 애니(App Annie) 등 앱 분석 서비스를 통한 트렌드 파악
        - 라이더 설문조사 (설문 응대 시 성실라이더 뱃지 보상 제공)
2. 리뉴얼의 목적
	- 국내 경쟁업체 라이더용 앱 1:1 대조 기능/외형 분석 (빅3 우선)
	- 국외 배달업체 라이더용 앱 1:1 대조 기능/외형 분석 (중국, 미국, 유럽 각 지역 빅1 우선)
	- 라이더들의 앱 사용패턴 분석 (현재 없는 로그의 경우 개발팀에 요청)
		- 배달 건수 대비 접속 횟수, 최다 사용 기능, 1회 접속 시 머무는 시간 등
	- 전체 접속 횟수 중 다운 비율, 바로 직전 실행된 기능, 보여진 화면 파악
	- 배달의민족 앱 사용자 대상 배민라이더스 앱 사용 후기 수집 (보상으로 포인트 제공)
	- 배민라이더스 라이더 대상 배달의민족 앱 사용 후기 수집 (보상으로 포인트 제공)
3. 리뉴얼 없이 개선
	- 전체 라이더 대비 앱 사용 라이더 비율 측정
	- 앱 사용 라이더 중 적극 사용 라이더 비율 측정
	- 비적극 라이더 대상 앱 사용 패턴 분석
	- 비사용 라이더 대상 배달 시 이용하는 다른 앱 조사 및 그 패턴 분석 (설문)
	- 라이더들의 최적 배달 동선을 '가장 빨리 맛집 찾길'이란 이름의 오픈데이터로 공개하는 방안 검토
	- B마트 주문이력 중 음식 관련 데이터를 통해 한식러/양식러/일식러로 고객을 분류
        - 주문자 가까이 위치한 라이더와 한식/양식/일식 음식점 사장님들에게 '각식러 분포 지도'를 공유하는 방안 검토
	- '가장 빨리 맛집 찾길' 및 '각식러 분포 지도' 적용 전 비용 대비 효과를 예측하여 앱 리뉴얼보다 효율이 클 경우
	    - 앱 리뉴얼 전 우선 적용 > 결과 측정 > 서비스 개선 여부 확인 > 앱 리뉴얼 진행 여부 재판단 (기준 성과 및 앱 환경 일부가 바뀌었으므로)

## [조사/분석 실행 시 유의사항]

1. 리뉴얼의 필요성
	- 라이더 만족도 조사 시 앱 리뉴얼 계획을 알리지 않고 비정기 설문조사 형태로 의뢰 (새 것에 대한 막연한 반발/기대 차단)
    - 경쟁업체 앱 만족도 분석 시 각 설문자의 소속에 따라 세부항목의 가중치를 다르게 적용 (이해관계에 따른 선입견 최소화)
	- 일반 앱 만족도 조사 시 라이더의 개인정보와 관련된 앱은 제외 (사생활 보호)
2. 리뉴얼의 목적
	- 국외 배달업체 라이더용 앱 분석 대행자 선정 시 고려 사항: 라이더 경험 필수, 체크리스트 제공
	- 배달의민족 앱 사용자의 배민라이더스 후기 수집 전, 해당 고객의 앱 내 접근범위 설정 필요
	- 배민라이더스 라이더가 배달의민족 앱 사용 시 관련자(예: 친한 사장님)를 통하지 않도록 사전 경고
3. 리뉴얼 없이 개선
	- 앱 비적극 사용 라이더의 다른 앱에 대한 솔직한 답변을 얻기 위해, 비사용에 따른 불이익이 없고 있다 해도 소급하여 적용되지 않음을 공지
	- 라이더 배달 동선 공개 결정 시 (소셜 엔지니어링을 적용하더라도) 라이더 개인의 식별이 불가하도록 전처리
	- 각식러 분포도 공유 결정 시 개별 고객의 식별이 불가하도록 전처리하되, 각각의 한식/양식/일식 선호 정도는 구체화 및 시각화하여 한 눈에 볼 수 있게 제공

## [중간 해결안 분석]

1. 리뉴얼 진행 여부
	- 진행 근거
        - 과거버전/경쟁앱/일반앱 (중 2개 항목 이상) 대비 80% 이하의 사용자 만족도
        - 3개 이상의 주요 기능이 부재하거나 50% 이상의 확률로 오작동
	- 비진행 근거
        - 과거버전/경쟁앱/일반앱 (중 2개 항목 이상) 대비 120% 이상의 만족도
        - 3개 이상의 일반 기능이 부재하거나 50% 이상의 확률로 오작동하지만, 주문 접수/배달 동선/결제 등의 주요 기능에 영향을 끼치지 않음
2. 리뉴얼 진행 시, 최우선 과제 선정
	- 기능성 개선 근거
        - 3개 이상의 주요 기능이 부재하거나 50% 이상의 확률로 오작동
	- 외형성 개선 근거
        - 배달의민족 사용자의 배민라이더스 앱 사용 후기와 배민라이더스 라이더의 배달의민족 사용 후기 모두에서
            - 배민라이더스 앱의 배달의민족 앱과의 외형 차이에 대한 부정적 응답이 50% 이상
                > 예: 배민라이더스 앱이 훨씬 덜 이뻐요!, 배민 앱 안 같아요!
3. 리뉴얼 비진행 시, 최우선 과제 선정
	- 가장 빨리 맛집 찾길: 배민라이더스 맛집 배달동선 오픈데이터화 근거
        - 리뉴얼 없이 데이터 연결이 가능한 경우
        - 라이더 개인 식별이 불가한 경우
        - 검색 기록 등을 통해 맛집 찾기에 대한 수요가 확인된 지역이 서울 구 기준 30% 이상일 때 (본 지역들 우선 적용)
	- 우리 동네 각식러 분포도: B마트 주문 기록을 맛집 사장님과 배민라이더스 라이더에게 공유하는 근거
        - 뚜렷한 분류가 가능할만큼의 데이터가 쌓인 경우
        - 고객 개인 식별이 불가한 경우
        - 공유 결정 시, 배민라이더스를 최근 30일 간 3회 이상 이용한 B마트 고객의 구매 이력을 분포도 작성에 우선 적용

## [각 해결안에 필요한 추가 조사 및 근거 확보]

1. 실행 단계 진입 전 사내 설문조사 실시
    - '리뉴얼, 안뉴얼, 딴 거 하고 리뉴얼' 각 안에 대한 사업부별 직군별 직책별 의견 수렴
2.  배민라이더스 외 사업부의 기존 데이터를 통해 해결가능한 부분이 있는지 파악 후 협조 가능 여부 확인

3.  특히 '딴 거 하고 리뉴얼'의 경우 B마트와 배달의민족 전사 브랜드팀과 사전 협의 필수

## [최종 해결안 선정]

1. 3개 후보안의 심층 조사를 통해 가장 타당한 근거를 갖춘 1개 후보를 선정

2. 초기 조사로 파악된 내용과 추가 조사의 내용이 불일치하거나 그 사이 변화된 부분은 없는지 점검

3. 잠정 선정된 해결안을 문서화하여 우선 보고

4. 관련자들을 모아 프레젠테이션 형태로 구두 제안 (화상회의도 가능)

5. 관련자 50% 이상의 응원(실은 동의) 하에 최종안 선정

## [최종 해결안 실행]

1. 실행 중 유의사항을 참고하며 지속적으로 실행의 정확도 및 예상 외 변수 파악

2. 모니터링 중 필요하다고 판단되거나 실행자로부터 요청이 들어오는 경우 추가 조사 및 분석

3. 추가 분석 결과 변경해야 할 실행 사항이 있다면 실행자에게 전달 후 적용 확인

## [결과 확인 및 평가]

1. 최우선 성과 확인 및 분석
	- 최종 목표를 달성하였는가? 비용은 얼마나 들었는가? 예측 범위 내인가?
	- 최우선 과제를 해결하였는가? 비용은 얼마나 들었는가? 예측 범위 내인가?
	- 최우선 과제를 달성하였지만 최종 목표는 달성하지 못했다면 그 이유는 무엇인가?
		- 목표에 맞는 과제 선정에 대한 의문
	- 이후 다른 업무를 위해 위의 문제를 어떻게 해결할 수 있는가?
		- 목표에 맞는 과제 선정 및 실행 전 검증을 위한 프로세스 확립
2. 추가적 성과 확인 및 분석
	- 기대하였고 실제로 달성된 성과는 무엇인가?
	- 기대하지 않았지만 달성된 성과는 무엇인가?
	- 추가 성과가 없다면 그 이유는 무엇인가? 
		- 제시된 해결안의 범용성, 확장성, 연계성에 대한 의문
	- 추가 성과, 연계 성과, 의외의 성과를 달성하기 위해 평소 어떤 활동이 필요한가?
		- 사업부, 직군, 직책 간 연계된 데이터 및 각각 진행 중인 프로젝트에 대한 의견 공유를 활성화할 수 있는 방안 마련
3. (계획대로, 하지만 온갖 변수에 따른 고난의 갱신 끝에) 최종 목표, 최우선 과제, 기대 성과, 추가 성과를 모두 달성한 경우
	- 칭찬 받아 마땅하고
	- 축하 받아 마땅하고
	- 인정(ㄷ...) 받아 마땅합니다!

In [ ]:
# 위 내용과 관련하여 질문이 있다면 누구든지 주저말고 아래 댓글로 달아주세요
## 그동안 kaggle 내에서 숨어 살았는데 이번 기회(?)에 처음으로 이렇게 공개 활동을 해보네요
### 워낙 고수분들이 많은 프로들의 세계라 겁도 나지만 설레기도 합니다
#### 긴 글 읽느라 고생 많으셨습니다. 따뜻한 연말 보내세요